<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/074_Environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



You're used to **tools** or **functions** doing things directly. But now we’re introducing the idea of an **Environment** as an **orchestrator** — a central hub that knows how to:

* look up available tools,
* manage execution context (memory, auth, LLM, etc.),
* inject dependencies automatically,
* and coordinate the execution of actions safely and consistently.

---

### 🧠 Think of the **Environment** as:

> **"The system that decides how and when things happen — not the tools themselves."**

It’s like a **conductor in an orchestra**, while the tools are the instruments.

---

## ✅ What the Environment Does (Step-by-Step)

Let’s walk through it:

```python
# A user input leads the agent to choose a tool and args
tool = tool_registry.get_tool(tool_name)
args = {...}
```

But instead of just calling:

```python
result = tool(**args)  # ⚠️ this is risky — no safety, no injection
```

The environment steps in:

```python
result = environment.execute_tool(tool, args, action_context)
```

---

### 🔧 What happens inside `environment.execute_tool(...)`

Here's what that execution typically involves:

```python
def execute_tool(self, tool, args, action_context):
    args_copy = args.copy()

    # 🔍 1. Check tool signature for special injected params
    if has_named_parameter(tool, "action_context"):
        args_copy["action_context"] = action_context

    if has_named_parameter(tool, "_auth_token"):
        args_copy["_auth_token"] = action_context.get("auth_token")

    if has_named_parameter(tool, "_llm"):
        args_copy["_llm"] = action_context.get("llm")

    # ✅ 2. Now call the tool with all the injected dependencies
    return tool(**args_copy)
```

---

### 🎯 What makes this powerful:

| Feature                    | Why It Matters                                               |
| -------------------------- | ------------------------------------------------------------ |
| ✅ Automatic injection      | Tool authors don’t need to manually wire up every dependency |
| ✅ Safer execution          | Central place to add logging, validation, rollback           |
| ✅ Scoped configuration     | The `ActionContext` can vary by request or agent             |
| ✅ Minimal tool boilerplate | Tools remain clean and reusable                              |

---

## 🧠 Think of the Environment as:

| Environment Role    | Analogy                                                                      |
| ------------------- | ---------------------------------------------------------------------------- |
| Tool executor       | Like an operating system calling an app with the right environment variables |
| Dependency injector | Like a factory wiring up your tool with config and memory                    |
| Safety layer        | Like a sandbox protecting unsafe or untrusted operations                     |
| Protocol enforcer   | Like an API gateway ensuring only allowed calls go through                   |

---

## 🔁 Optional: You can even add advanced features

* Transaction staging (like `StagedActionEnvironment`)
* Logging every tool usage
* Rate-limiting or access control
* Reversible actions with rollback
* Multi-agent coordination



---

## ✅ **How Does the Environment Take Action?**

You’re used to functions taking action. In this system, **the environment is the component that wraps function calls**, **prepares arguments**, and **decides what to pass**.

### ⚙️ How It Works

Let’s break down this line:

```python
args_copy = args.copy()
if has_named_parameter(tool, "_auth_token"):
    args_copy["_auth_token"] = action_context.get("auth_token")
```

This happens **inside the environment**, before a tool is executed.

So, say we have a tool like:

```python
def update_profile(username: str, _auth_token: str) -> dict:
    ...
```

The environment’s job is:

1. Check what parameters this tool needs.
2. See if any of them are **context-aware dependencies** (like `_auth_token`, `action_context`, `llm`, `memory`, etc.).
3. If yes, **pull them from the `ActionContext`** and add them to the call.

This makes tools feel like regular functions, even though they’re backed by dynamic context-aware logic.

---

### 🔄 Who *Runs* This Environment?

Usually, the **agent loop** or a **tool executor** owns this responsibility.

Example of a basic environment-driven executor:

```python
def run_tool(tool, action_context, args):
    args_copy = args.copy()
    if has_named_parameter(tool, "action_context"):
        args_copy["action_context"] = action_context
    if has_named_parameter(tool, "_auth_token"):
        args_copy["_auth_token"] = action_context.get("auth_token")
    
    return tool(**args_copy)
```

This **executor is part of the environment**, not the tool. The tool just defines what it needs — and the environment supplies it.

---

## ✅ Summary

| Concept              | Role                                                               |
| -------------------- | ------------------------------------------------------------------ |
| `ActionContext`      | Container for all runtime-specific resources (memory, LLM, tokens) |
| Tool                 | Stateless function that declares what it needs via parameters      |
| Environment/Executor | Middleware that **injects** context into tools automatically       |






## ✅ 1. What triggers `environment.execute_tool(...)`?

The **agent loop** is what triggers the environment to execute tools.

Here’s the big-picture flow:

```python
while True:
    # 1. Construct a prompt that describes the current task
    prompt = self.construct_prompt(...)

    # 2. LLM decides what tool to use and with what arguments
    response = self.prompt_llm_for_action(...)

    # 3. Agent passes the tool + args to the environment
    result = environment.execute_tool(tool, args, action_context)
```

🧠 **The agent itself is not running the tool directly.**
It’s deferring to the `environment` to *safely* execute the chosen tool.

So the **LLM decides what to do**, and the **environment decides how it gets done**.

---

## ✅ 2. Where does `environment` come from?

You're absolutely right — in the examples we’ve seen, `environment` seems to come out of nowhere. But in a real system, you'd explicitly instantiate it like this:

```python
environment = Environment()
```

Or if you're using a specialized one:

```python
environment = StagedActionEnvironment()  # e.g. if you want to stage reversible actions
```

Then you'd pass it into your agent when running it:

```python
agent.run(
    user_input="Update the project status",
    memory=my_memory,
    environment=environment,  # 👈 key injection point
    action_context_props={"auth_token": "abc123"}
)
```

Then inside the agent:

```python
result = environment.execute_tool(tool, args, action_context)
```

This gives the **agent complete flexibility** to:

* use different environments per request (e.g. dev vs prod),
* swap in staging, validation, or mocking,
* apply different rules per use case.

---

## 🔁 Summary of Control Flow

| Step          | Role                                                    |
| ------------- | ------------------------------------------------------- |
| User input    | Triggers the agent                                      |
| Agent loop    | Constructs prompt and receives LLM output (tool + args) |
| Environment   | Executes the tool with safe, injected dependencies      |
| ActionContext | Supplies those dependencies (memory, auth, LLM, etc.)   |



Let’s walk through a **minimal working example** that shows the full flow of:

* agent receives user input,
* LLM selects a tool and arguments (simulated),
* environment executes the tool,
* action context provides injected dependencies.

We'll use fake tools and a simulated LLM to keep it simple — just enough to **show the wiring** clearly.

---

## ✅ Step-by-Step Code: Environment + Agent + Tool

```python
import uuid
from typing import Dict, Callable

# -----------------------
# 1. ActionContext
# -----------------------

class ActionContext:
    def __init__(self, properties: Dict = None):
        self.context_id = str(uuid.uuid4())
        self.properties = properties or {}

    def get(self, key, default=None):
        return self.properties.get(key, default)

# -----------------------
# 2. Tool registry
# -----------------------

tool_registry = {}

def register_tool(fn: Callable):
    tool_registry[fn.__name__] = fn
    return fn

# -----------------------
# 3. Environment
# -----------------------

class Environment:
    def execute_tool(self, tool_name: str, args: dict, action_context: ActionContext):
        tool = tool_registry.get(tool_name)
        if not tool:
            raise ValueError(f"Tool '{tool_name}' not found")

        # Inject dependencies from action_context if the tool requires them
        args_copy = args.copy()
        if 'action_context' in tool.__code__.co_varnames:
            args_copy['action_context'] = action_context

        return tool(**args_copy)

# -----------------------
# 4. Tool
# -----------------------

@register_tool
def convert_to_uppercase(text: str, action_context: ActionContext = None) -> str:
    # Optional: use context info like user preferences, logging, etc.
    user = action_context.get("user") if action_context else "unknown"
    print(f"[Tool] User: {user} - Converting text to uppercase.")
    return text.upper()

# -----------------------
# 5. Agent
# -----------------------

class Agent:
    def __init__(self, environment: Environment):
        self.environment = environment

    def run(self, user_input: str, action_context_props: Dict = None):
        # Build ActionContext with dependencies
        action_context = ActionContext({
            "user": "alice",  # Injected dependency
            **(action_context_props or {})
        })

        # Simulated LLM parsing (we fake a tool call)
        tool_name = "convert_to_uppercase"
        args = {"text": user_input}

        print(f"[Agent] Calling tool: {tool_name} with args: {args}")
        result = self.environment.execute_tool(tool_name, args, action_context)

        print(f"[Agent] Tool result: {result}")
        return result

# -----------------------
# 6. Run the system
# -----------------------

if __name__ == "__main__":
    env = Environment()
    agent = Agent(env)

    # Run the agent with user input
    agent.run("hello world")
```

---

## ✅ Output (what you'd expect to see)

```
[Agent] Calling tool: convert_to_uppercase with args: {'text': 'hello world'}
[Tool] User: alice - Converting text to uppercase.
[Agent] Tool result: HELLO WORLD
```

---

## 🧠 What You Should Be Focusing On:

| Element         | Focus Point                                                        |
| --------------- | ------------------------------------------------------------------ |
| `ActionContext` | Flexible injection of things like auth token, memory, user, or LLM |
| `Environment`   | Executes tools using dependencies from context                     |
| `Agent`         | Interprets task (simulated), delegates execution to environment    |
| `Tool`          | Independent, testable, gets only what it needs via parameters      |




Let’s break down the `Environment` class **from the ground up**, assuming minimal prior exposure.

---

🧱 What is a Class in Python?

A `class` is a **blueprint** for creating reusable, structured code. It groups together **data** (attributes) and **behavior** (methods).

For example, `Environment` is a class that knows **how to run tools**.

---

## 🧭 What Is the Purpose of the `Environment` Class?

The `Environment` class is designed to:

### ✅ Centralize tool execution logic

Rather than having each agent know how to execute every tool (and how to inject dependencies like memory or auth tokens), we let `Environment` handle it.

### ✅ Handle dependency injection

It looks at what the tool needs (e.g., `action_context`, `auth_token`) and makes sure those things are passed in automatically.

### ✅ Keep agents clean

Agents don’t need to worry about how tools work — they just say *“Run this tool with these arguments.”*

---

## 🔍 What Does the `Environment` Contain?

Here’s a simplified version of the class:

```python
class Environment:
    def __init__(self, tool_registry):
        self.tool_registry = tool_registry  # A dict or registry of available tools

    def execute_tool(self, tool_name: str, args: dict, action_context: ActionContext):
        tool = self.tool_registry.get(tool_name)
        if not tool:
            raise ValueError(f"Tool not found: {tool_name}")

        # Automatically inject action_context if tool expects it
        if "action_context" in inspect.signature(tool).parameters:
            return tool(action_context=action_context, **args)
        else:
            return tool(**args)
```

### 🔨 `__init__`

Sets up the environment. Usually takes in the tool registry.

### 🧠 `execute_tool(...)`

* Looks up the tool by name.
* Inspects what the tool needs.
* Injects things like `action_context` if necessary.
* Executes the tool.

---

## 💡 Why Is This Powerful?

Here’s what this enables:

| Without Environment                    | With Environment                 |
| -------------------------------------- | -------------------------------- |
| Agents need to know all tool details   | Agents just call tools by name   |
| You repeat logic across agents         | Centralized, reusable logic      |
| Harder to test or replace dependencies | Easy to inject/test dependencies |
| Harder to maintain                     | Modular and clean                |

---

## 🔁 Real-World Analogy

Think of the `Environment` as a **universal remote control**. The agent pushes a button (“run this tool”), and the `Environment` knows:

* Where the tool is
* What it needs to work (batteries = auth\_token, memory, etc.)
* How to plug everything in and get it working

---

## ✅ In Summary

* `Environment` is a **tool execution manager**.
* It handles **dependency injection**, **tool lookup**, and **clean execution**.
* It separates concerns so **agents can focus on decisions**, and the **environment handles execution**.
* It makes your system **modular**, **scalable**, and **testable**.




## 🛠️ **The Environment as a Tool Shed**

> **Environment = portable, well-stocked tool shed.**

### 📦 What’s in the Shed?

| Tool Shed Component (Environment) | Real-World Tool Analogy        | Software Counterpart                                       |
| --------------------------------- | ------------------------------ | ---------------------------------------------------------- |
| Tools                             | Lawnmower, rake, hammer        | `Tool` functions (e.g. `send_email`, `check_availability`) |
| Dependencies                      | Gasoline, extension cord       | `auth_token`, `memory`, `llm`                              |
| Inventory System                  | Tool checkout list             | `tool_registry` (mapping tool names to their functions)    |
| Shed Manager                      | Person who hands you the tools | `Environment.execute_tool(...)`                            |
| Portable Design                   | Can drop it on any job site    | Works across dev/test/prod agents                          |

---

## 🧰 When a Tool Is Requested

Let’s say an agent says:

> “I want to use the **calendar invite tool**.”

The `Environment`:

1. Looks it up in the **tool registry**.
2. Checks the **function signature**:

   * Does it need memory? ✅
   * Does it need an API key? ✅
   * Does it expect `action_context`? ✅
3. Gathers all required parts:

   * Pulls them from its internal store or from `action_context`.
4. Runs the tool with everything neatly **injected and wired up**.

---

## 🪄 Why This Matters

* 🔄 **Reusable**: You don’t hardcode tools to one agent.
* 🧪 **Testable**: You can mock tools or contexts independently.
* 🧱 **Composable**: You can build new workflows just by changing which agent uses which tool from the shed.
* 🧳 **Portable**: You can drop the same environment in another app, server, or runtime and it works.

---

## 🧠 Final Thought

The real beauty is this:

> **You’re not just building tools. You’re building an ecosystem.**

An ecosystem where:

* Tools are **standardized**.
* The environment is **centralized**.
* Agents are **smart dispatchers**, not electricians.



Here's a **simple, complete working example** that wires together:

* ✅ A **tool** (e.g. `send_email`)
* ✅ An **agent** that uses that tool
* ✅ An **environment** that manages execution
* ✅ A basic **ActionContext** with dependencies

---

## 🧪 Use Case

We'll simulate an agent that takes a user prompt like *"Please email the team"* and uses the `send_email` tool to do it.

---

### ✅ 1. Define a Simple Tool

```python
from typing import List

def send_email(action_context, to: List[str], subject: str, body: str) -> dict:
    smtp = action_context.get("smtp_service")
    # Fake send logic for demo purposes
    print(f"[📤 Sending email] To: {to}, Subject: {subject}")
    return {"status": "sent", "recipients": to}
```

---

### ✅ 2. Create a Tool Registry and Register the Tool

```python
class ToolRegistry:
    def __init__(self):
        self.tools = {}

    def register_tool(self, name, func):
        self.tools[name] = func

    def get_tool(self, name):
        return self.tools.get(name)
```

```python
tool_registry = ToolRegistry()
tool_registry.register_tool("send_email", send_email)
```

---

### ✅ 3. Define the `ActionContext`

```python
class ActionContext:
    def __init__(self, properties=None):
        self.properties = properties or {}

    def get(self, key, default=None):
        return self.properties.get(key, default)
```

---

### ✅ 4. Define the `Environment`

```python
class Environment:
    def __init__(self, tool_registry, action_context):
        self.tool_registry = tool_registry
        self.action_context = action_context

    def execute_tool(self, tool_name, args):
        tool = self.tool_registry.get_tool(tool_name)
        if not tool:
            raise ValueError(f"Tool '{tool_name}' not found")

        return tool(self.action_context, **args)
```

---

### ✅ 5. Create a Simple Agent That Uses the Tool

```python
class Agent:
    def __init__(self, environment):
        self.environment = environment

    def run(self, user_input: str):
        if "email" in user_input.lower():
            args = {
                "to": ["team@example.com"],
                "subject": "Team Update",
                "body": "Hello team, just a quick update!"
            }
            result = self.environment.execute_tool("send_email", args)
            print(f"[Agent] Tool result: {result}")
        else:
            print("[Agent] No action taken.")
```

---

### ✅ 6. Wire It All Together and Run

```python
# Create dependencies
smtp_service = "DummySMTP"  # This would be your real email service
context = ActionContext({"smtp_service": smtp_service})

# Create the environment
env = Environment(tool_registry, context)

# Create the agent
agent = Agent(env)

# Run the agent with a user input
agent.run("Can you please email the team?")
```

---

### 🧠 What’s Happening?

| Layer           | Responsibility                                             |
| --------------- | ---------------------------------------------------------- |
| **Tool**        | Defines a single atomic capability (`send_email`)          |
| **Context**     | Injects dependencies (`smtp_service`)                      |
| **Environment** | Looks up and executes the right tool with args and context |
| **Agent**       | Decides what tool to use based on user input               |


